<a href="https://colab.research.google.com/github/priyanshueeeee/Metron-LSTM/blob/main/Metron-LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
df = pd.read_csv('/content/Dataset.csv')

In [ ]:
df.head()
df.info()
data_wo_dates = df.iloc[:, 1:].values
data_wo_dates
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2042 entries, 0 to 2041
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2042 non-null   object 
 1   CDX.NA.HY  2042 non-null   float64
 2   CDX.NA.IG  2042 non-null   float64
 3   SPOT       2042 non-null   float64
 4   VALUE      2042 non-null   float64
 5   Vol        2042 non-null   float64
 6   EQU        2042 non-null   float64
dtypes: float64(6), object(1)
memory usage: 111.8+ KB


In [ ]:
df.head()

,Date,CDX.NA.HY,CDX.NA.IG,SPOT,VALUE,Vol,EQU
0,01/17/2020,109.630,44.235,1.6195,1.657563,12.10,3329.62
1,01/16/2020,109.599,44.176,1.6247,1.656338,12.32,3316.81
2,01/15/2020,109.515,44.909,1.5985,1.639625,12.42,3289.29
3,01/14/2020,109.539,44.897,1.6199,1.660338,12.39,3283.15
4,01/13/2020,109.651,44.350,1.6495,1.682400,12.32,3288.13


In [ ]:
import numpy as np

def split_dataset(dataset):
    """
    Splits the dataset into X (all columns except the first) and Y (the first column).

    Args:
        dataset: The input dataset (NumPy array or list).

    Returns:
        X: The features (all columns except the first).
        Y: The target variable (the first column).
    """
    X = dataset[:, 1:]
    Y = dataset[:, 0]
    return X, Y

In [ ]:
import numpy as np

def create_dataset(X, Y, look_back=1, horizon=1):
    """
    Creates a dataset for time series forecasting with lookback and horizon.

    Args:
        X: The input features (already split).
        Y: The target variable (already split).
        look_back: The number of past time steps to consider.
        horizon: The number of future time steps to predict.

    Returns:
        X_dataset: The input dataset for forecasting.
        Y_dataset: The target dataset for forecasting.
    """
    X_dataset, Y_dataset = [], []
    for i in range(len(X) - look_back - horizon + 1):
        X_dataset.append(X[i:(i + look_back)])
        Y_dataset.append(Y[i + look_back: i + look_back + horizon])
    return np.array(X_dataset), np.array(Y_dataset)

In [ ]:
X, Y = split_dataset(data_wo_dates)
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (2042, 5)
Y shape: (2042,)


In [ ]:
look_back = 10
horizon = 5

X_dataset, Y_dataset = create_dataset(X, Y, look_back, horizon)

In [ ]:
split_index = int(len(X_dataset) * 0.7)
X_train = X_dataset[:split_index]
X_test = X_dataset[split_index:]
Y_train = Y_dataset[:split_index]
Y_test = Y_dataset[split_index:]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (1419, 10, 5)
X_test shape: (609, 10, 5)
Y_train shape: (1419, 5)
Y_test shape: (609, 5)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1419, 10, 5)
(609, 10, 5)
(1419, 5)
(609, 5)


In [ ]:
print(X_dataset)

[[[4.4235000e+01 1.6195000e+00 1.6575625e+00 1.2100000e+01 3.3296200e+03]
  [4.4176000e+01 1.6247000e+00 1.6563375e+00 1.2320000e+01 3.3168100e+03]
  [4.4909000e+01 1.5985000e+00 1.6396250e+00 1.2420000e+01 3.2892900e+03]
  ...
  [4.5290000e+01 1.6629000e+00 1.6905500e+00 1.3450000e+01 3.2530500e+03]
  [4.6865000e+01 1.6122000e+00 1.6443875e+00 1.3790000e+01 3.2371800e+03]
  [4.6375000e+01 1.6058000e+00 1.6436875e+00 1.3850000e+01 3.2462800e+03]]

 [[4.4176000e+01 1.6247000e+00 1.6563375e+00 1.2320000e+01 3.3168100e+03]
  [4.4909000e+01 1.5985000e+00 1.6396250e+00 1.2420000e+01 3.2892900e+03]
  [4.4897000e+01 1.6199000e+00 1.6603375e+00 1.2390000e+01 3.2831500e+03]
  ...
  [4.6865000e+01 1.6122000e+00 1.6443875e+00 1.3790000e+01 3.2371800e+03]
  [4.6375000e+01 1.6058000e+00 1.6436875e+00 1.3850000e+01 3.2462800e+03]
  [4.5973000e+01 1.5896000e+00 1.6234125e+00 1.4020000e+01 3.2348500e+03]]

 [[4.4909000e+01 1.5985000e+00 1.6396250e+00 1.2420000e+01 3.2892900e+03]
  [4.4897000e+01 1.619

In [ ]:
print(X_train)

[[[4.4235000e+01 1.6195000e+00 1.6575625e+00 1.2100000e+01 3.3296200e+03]
  [4.4176000e+01 1.6247000e+00 1.6563375e+00 1.2320000e+01 3.3168100e+03]
  [4.4909000e+01 1.5985000e+00 1.6396250e+00 1.2420000e+01 3.2892900e+03]
  ...
  [4.5290000e+01 1.6629000e+00 1.6905500e+00 1.3450000e+01 3.2530500e+03]
  [4.6865000e+01 1.6122000e+00 1.6443875e+00 1.3790000e+01 3.2371800e+03]
  [4.6375000e+01 1.6058000e+00 1.6436875e+00 1.3850000e+01 3.2462800e+03]]

 [[4.4176000e+01 1.6247000e+00 1.6563375e+00 1.2320000e+01 3.3168100e+03]
  [4.4909000e+01 1.5985000e+00 1.6396250e+00 1.2420000e+01 3.2892900e+03]
  [4.4897000e+01 1.6199000e+00 1.6603375e+00 1.2390000e+01 3.2831500e+03]
  ...
  [4.6865000e+01 1.6122000e+00 1.6443875e+00 1.3790000e+01 3.2371800e+03]
  [4.6375000e+01 1.6058000e+00 1.6436875e+00 1.3850000e+01 3.2462800e+03]
  [4.5973000e+01 1.5896000e+00 1.6234125e+00 1.4020000e+01 3.2348500e+03]]

 [[4.4909000e+01 1.5985000e+00 1.6396250e+00 1.2420000e+01 3.2892900e+03]
  [4.4897000e+01 1.619

In [ ]:
print(Y_test)

[[107.031 106.902 106.971 106.81  107.09 ]
 [106.902 106.971 106.81  107.09  107.128]
 [106.971 106.81  107.09  107.128 107.286]
 ...
 [ 88.689  89.062  89.997  89.948  90.521]
 [ 89.062  89.997  89.948  90.521  91.084]
 [ 89.997  89.948  90.521  91.084  91.429]]


In [ ]:
print(X_train[0][0])

[4.4235000e+01 1.6195000e+00 1.6575625e+00 1.2100000e+01 3.3296200e+03]


In [ ]:
print(X_train.shape[1])

10


In [ ]:
print(Y_train[0][0])

109.569


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train.reshape(X_train.shape[0], -1))
Y_train_scaled = scaler_Y.fit_transform(Y_train.reshape(Y_train.shape[0], -1))

X_train_scaled = X_train_scaled.reshape(X_train.shape)
Y_train_scaled = Y_train_scaled.reshape(Y_train.shape)

print("X_train_scaled shape:", X_train_scaled.shape)
print("Y_train_scaled shape:", Y_train_scaled.shape)

X_train_scaled shape: (1419, 10, 5)
Y_train_scaled shape: (1419, 5)


In [ ]:
print(X_train_scaled)

[[[0.00252865 0.3163199  0.36159516 0.09367089 1.        ]
  [0.00180089 0.31873491 0.36104034 0.10063291 1.        ]
  [0.01084235 0.30656697 0.35347102 0.10379747 1.        ]
  ...
  [0.01393642 0.33647594 0.37653566 0.13639241 0.99664047]
  [0.03339552 0.31292959 0.35562802 0.1471519  0.98553301]
  [0.02734158 0.30995727 0.35531098 0.14905063 0.99190213]]

 [[0.00180089 0.31873491 0.36104034 0.10063291 0.99146307]
  [0.01084235 0.30656697 0.35347102 0.10379747 0.98150202]
  [0.01069433 0.31650567 0.362852   0.1028481  0.99579513]
  ...
  [0.03339552 0.31292959 0.35562802 0.1471519  0.98553301]
  [0.02734158 0.30995727 0.35531098 0.14905063 0.99190213]
  [0.02237487 0.30243359 0.34612815 0.15443038 0.98390224]]

 [[0.01084235 0.30656697 0.35347102 0.10379747 0.97312301]
  [0.01069433 0.31650567 0.362852   0.1028481  0.97737493]
  [0.00394716 0.33025265 0.37284441 0.10063291 0.99920559]
  ...
  [0.02734158 0.30995727 0.35531098 0.14905063 0.99190213]
  [0.02237487 0.30243359 0.3461281

In [ ]:
X_test_scaled = scaler_X.transform(X_test.reshape(X_test.shape[0], -1))
Y_test_scaled = scaler_Y.transform(Y_test.reshape(Y_test.shape[0], -1))

X_test_scaled = X_test_scaled.reshape(X_test.shape)
Y_test_scaled = Y_test_scaled.reshape(Y_test.shape)

print("X_test_scaled shape:", X_test_scaled.shape)
print("Y_test_scaled shape:", Y_test_scaled.shape)

X_test_scaled shape: (609, 10, 5)
Y_test_scaled shape: (609, 5)


In [ ]:
print(Y_test_scaled)

[[ 0.78890463  0.77928566  0.78771996  0.77566455  0.79663048]
 [ 0.77928566  0.78443069  0.77566455  0.79663048  0.79947585]
 [ 0.78443069  0.77242562  0.79663048  0.79947585  0.81130663]
 ...
 [-0.57877861 -0.55096563 -0.48326469 -0.48693373 -0.44402845]
 [-0.55096563 -0.48124674 -0.48693373 -0.44402845 -0.40187196]
 [-0.48124674 -0.48490045 -0.44402845 -0.40187196 -0.37603894]]


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

model = keras.Sequential()
model.add(layers.LSTM(units=300, return_sequences=True, input_shape=(X_train.shape[1], 5)))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units=300))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=horizon))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01, decay=0.01 / 125)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(X_train_scaled, Y_train_scaled, epochs=250, batch_size=64)
predictions = model.predict(X_test_scaled)

Epoch 1/250


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 112ms/step - loss: 0.5216
Epoch 2/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 174ms/step - loss: 0.0492
Epoch 3/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - loss: 0.0402
Epoch 4/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 108ms/step - loss: 0.0382
Epoch 5/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - loss: 0.0382
Epoch 6/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - loss: 0.0323
Epoch 7/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - loss: 0.0335
Epoch 8/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 108ms/step - loss: 0.0362
Epoch 9/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - loss: 0.0335
Epoch 10/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - loss: 0.0309
Epoch 11/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - loss: 0.0283
Epoch 12/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - loss: 0.0243
Epoch 13/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - loss: 0.0178
Epoch 14/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - loss: 0.0218
Epoch 15/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step -

In [ ]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test_scaled, predictions)
print(rmse)

0.24656926241979887


In [ ]:
print(len(predictions))

609


In [ ]:
print(predictions)

[[0.8066658  0.80731684 0.812411   0.8073771  0.81200993]
 [0.82461363 0.82495284 0.8284006  0.82297957 0.8270382 ]
 [0.8309793  0.83123815 0.8348857  0.8295934  0.8339478 ]
 ...
 [0.24325448 0.23511887 0.23659444 0.24426931 0.2615534 ]
 [0.2431562  0.2350418  0.23651135 0.24421346 0.26150423]
 [0.24306312 0.23497474 0.23644239 0.24417108 0.26146403]]


In [ ]:
print(predictions.shape)

(609, 5)


In [ ]:
predictions_reshaped = predictions.reshape(predictions.shape[0], -1)
original_predictions = scaler_Y.inverse_transform(predictions_reshaped)
original_predictions = original_predictions.reshape(predictions.shape)
print("Original predictions shape:", original_predictions.shape)

Original predictions shape: (609, 5)


In [ ]:
print(original_predictions)

[[107.269196 107.27793  107.30075  107.23353  107.29539 ]
 [107.50989  107.51444  107.51429  107.441895 107.4961  ]
 [107.59526  107.59874  107.60089  107.53022  107.58837 ]
 ...
 [ 99.71329   99.60418   99.61072   99.71321   99.944046]
 [ 99.71197   99.60315   99.60961   99.71247   99.94339 ]
 [ 99.710724  99.60224   99.60869   99.7119    99.94285 ]]


In [ ]:
print(Y_test)

[[107.031 106.902 106.971 106.81  107.09 ]
 [106.902 106.971 106.81  107.09  107.128]
 [106.971 106.81  107.09  107.128 107.286]
 ...
 [ 88.689  89.062  89.997  89.948  90.521]
 [ 89.062  89.997  89.948  90.521  91.084]
 [ 89.997  89.948  90.521  91.084  91.429]]


In [ ]:
from sklearn.metrics import root_mean_squared_error
rmse1= root_mean_squared_error(Y_test, original_predictions)
print(rmse1)

3.298347955559437
